## Libraries

In [15]:
!pip install beautifulsoup4
import requests
import json
from bs4 import BeautifulSoup
import re

## SQLite operations functions

In [16]:
import sqlite3

def create_table():
    """
    Creates the 'movies' table in the SQLite database if it doesn't already exist.
    The table includes columns for id, title, release_year, director, box_office, and country.
    """
    # Step 1: Connect to the SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Step 2: Define the SQL command to create the table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS movies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER NOT NULL,
        director TEXT,
        box_office TEXT,
        country TEXT
    );
    """

    # Step 3: Execute the SQL command to create the table
    cursor.execute(create_table_query)

    # Step 4: Commit the changes to the database
    conn.commit()

    # Step 5: Verify the table creation by querying the database schema
    cursor.execute("PRAGMA table_info(movies)")
    columns = cursor.fetchall()
    print("Table Schema:")
    for column in columns:
        print(column)

    # Step 6: Close the database connection
    conn.close()


def delete():
    """
    Deletes all records from the 'movies' table.
    """
    # Step 1: Connect to the SQLite database
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Step 2: Define the SQL command to delete all records
    delete_query = "DELETE FROM movies;"

    # Step 3: Execute the SQL command
    cursor.execute(delete_query)

    # Step 4: Commit the changes to the database
    conn.commit()

    # Step 5: Close the database connection
    conn.close()


def select(save_to_json=False, json_filename='movies.json'):
    """
    Retrieves all records from the 'movies' table and returns them as a list of dictionaries.
    Optionally saves the data to a JSON file.

    Args:
        save_to_json (bool): If True, saves the data to a JSON file.
        json_filename (str): The name of the JSON file to save the data.

    Returns:
        list: A list of dictionaries containing movie data.
    """
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    select_query = "SELECT * FROM movies;"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    # Convert the rows into a list of dictionaries
    movies = [
        {
            'title': row[1],
            'revenue': row[4],
            'release_year': row[2],
            'director': row[3],
            'country': row[5]
        }
        for row in rows
    ]

    conn.close()

    # Save the data to a JSON file if requested
    if save_to_json:
        with open(json_filename, 'w') as json_file:
            json.dump(movies, json_file, indent=4)  # Use indent for pretty formatting
        print(f"Data saved to {json_filename}")

    return movies


def insert(title, release_year, director, box_office, country):
    """
    Inserts a new movie record into the 'movies' table.

    Args:
        title (str): The title of the movie.
        release_year (int): The release year of the movie.
        director (str): The director(s) of the movie.
        box_office (str): The box office revenue of the movie.
        country (str): The country of origin of the movie.
    """
    # Step 1: Connect to the SQLite database
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Step 2: Define the SQL command to insert a record
    insert_query = """
    INSERT INTO movies (title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?);
    """

    # Step 3: Execute the SQL command with parameters to avoid SQL injection
    cursor.execute(insert_query, (title, release_year, director, box_office, country))

    # Step 4: Commit the changes to the database
    conn.commit()

    # Step 5: Close the database connection
    conn.close()

## Wikipedia parsing

In [17]:
# Fetch the main page containing the list of highest-grossing films
main_page_url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
main_page_soup = BeautifulSoup(requests.get(main_page_url).content, 'html.parser')

# Initialize a list to store movie data
movies = []

# Find the table containing the movie data
movie_table = main_page_soup.find('table', class_='wikitable')

# Skip the first row (header) and iterate over the remaining rows
for row in movie_table.find_all('tr')[1:]:
    # Extract the movie title and its Wikipedia link
    movie_title_element = row.find('th').find('a')
    movie_title = movie_title_element.get('title')
    movie_url = 'https://en.wikipedia.org' + movie_title_element.get('href')

    # Fetch the individual movie page
    movie_page_soup = BeautifulSoup(requests.get(movie_url).content, 'html.parser')

    # Extract the director(s) information
    director_element = movie_page_soup.find('td', class_='infobox-data')
    if director_element:
        director_text = director_element.text.strip()
        # Clean up director names (remove citations like [1], [2], etc.)
        directors = [d.split('[')[0].strip() for d in director_text.split('\n')]
        director = ', '.join(directors)
    else:
        director = 'Unknown'

    # Extract the country information
    country_element = movie_page_soup.find('th', string=re.compile('Countr(?:y|ies)'))
    if country_element:
        country_value = country_element.find_parent('tr').find('td').text.strip()
        # Clean up country names (remove citations like [1], [2], etc.)
        countries = [c.split('[')[0].strip() for c in re.split('\n', country_value)]
        country = ', '.join(countries)
    else:
        country = 'Unknown'

    # Extract revenue and release year from the main table
    revenue = list(row.find_all('td')[2].children)[-1].text.strip()
    release_year = row.find_all('td')[3].text.strip()

    # Append the movie data to the list
    movies.append({
        'title': movie_title,
        'revenue': revenue,
        'release_year': release_year,
        'director': director,
        'country': country
    })

# Print the extracted movie data
print(movies)

# Perform database operations
create_table()
delete()
for movie in movies:
    insert(movie['title'], movie['release_year'], movie['director'], movie['revenue'], movie['country'])

[{'title': 'Avatar (2009 film)', 'revenue': '$2,923,706,026', 'release_year': '2009', 'director': 'James Cameron', 'country': 'United Kingdom, United States'}, {'title': 'Avengers: Endgame', 'revenue': '$2,797,501,328', 'release_year': '2019', 'director': 'Anthony RussoJoe Russo', 'country': 'United States'}, {'title': 'Avatar: The Way of Water', 'revenue': '$2,320,250,281', 'release_year': '2022', 'director': 'James Cameron', 'country': 'United States'}, {'title': 'Titanic (1997 film)', 'revenue': '$2,257,844,554', 'release_year': '1997', 'director': 'James Cameron', 'country': 'United States'}, {'title': 'Star Wars: The Force Awakens', 'revenue': '$2,068,223,624', 'release_year': '2015', 'director': 'J. J. Abrams', 'country': 'United States'}, {'title': 'Avengers: Infinity War', 'revenue': '$2,048,359,754', 'release_year': '2018', 'director': 'Anthony RussoJoe Russo', 'country': 'United States'}, {'title': 'Spider-Man: No Way Home', 'revenue': '$1,922,598,800', 'release_year': '2021'

## Saving to JSON

In [18]:
select(save_to_json=True, json_filename='data.json')

Data saved to data.json


[{'title': 'Avatar (2009 film)',
  'revenue': '$2,923,706,026',
  'release_year': 2009,
  'director': 'James Cameron',
  'country': 'United Kingdom, United States'},
 {'title': 'Avengers: Endgame',
  'revenue': '$2,797,501,328',
  'release_year': 2019,
  'director': 'Anthony RussoJoe Russo',
  'country': 'United States'},
 {'title': 'Avatar: The Way of Water',
  'revenue': '$2,320,250,281',
  'release_year': 2022,
  'director': 'James Cameron',
  'country': 'United States'},
 {'title': 'Titanic (1997 film)',
  'revenue': '$2,257,844,554',
  'release_year': 1997,
  'director': 'James Cameron',
  'country': 'United States'},
 {'title': 'Star Wars: The Force Awakens',
  'revenue': '$2,068,223,624',
  'release_year': 2015,
  'director': 'J. J. Abrams',
  'country': 'United States'},
 {'title': 'Avengers: Infinity War',
  'revenue': '$2,048,359,754',
  'release_year': 2018,
  'director': 'Anthony RussoJoe Russo',
  'country': 'United States'},
 {'title': 'Spider-Man: No Way Home',
  'revenu